In [1]:
localaddress = "http://YOUR_IP_ADDRESS:PORT"

In [ ]:
from selenium import webdriver
from selenium.webdriver.edge.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pickle
import time
import os

def save_cookies_for_trendinsight():
    """保存Trendinsight网站的Cookie"""
    
    # 初始化浏览器选项
    options = Options()
    options.add_argument('--disable-blink-features=AutomationControlled')
    options.add_argument('--start-maximized')
    
    try:
        # 连接到远程EdgeDriver
        driver = webdriver.Remote(
            command_executor=localaddress,
            options=options
        )
        
        print("✅ 浏览器初始化成功")
        print("🌐 正在访问Trendinsight网站...")
        
        # 访问目标网站
        target_url = "https://example.com"
        driver.get(target_url)
        
        # 检查当前页面状态
        current_url = driver.current_url
        time.sleep(10)
        print(f"🔗 当前URL: {current_url}")
        
        # 判断是否需要登录
        # ✅ 改为 DOM 特征判断：只要找不到 ZhuiTable-tableBody 就认为是登录页
        if not driver.find_elements(By.CLASS_NAME, "ZhuiTable-tableBody"):
            print("🔐 检测到登录页面，请手动完成登录...")
            print("💡 请在弹出的浏览器窗口中完成登录操作")
            print("⏳ 等待登录完成...")
            
            # 等待用户手动登录（最长等待5分钟）
            login_timeout = 180  # 5分钟
            start_time = time.time()
            
            while time.time() - start_time < login_timeout:
                current_url = driver.current_url
                # 检查是否跳转到了目标页面（登录成功）
                if "my-subscript" in current_url and "login" not in current_url.lower():
                    print("✅ 登录成功！检测到目标页面")
                    break
                
                # 检查是否仍在登录页面
                if "login" in current_url.lower() or "signin" in current_url.lower():
                    print("⏳ 仍在登录页面，请继续完成登录...")
                else:
                    print(f"🔄 当前页面: {current_url}")
                
                time.sleep(100)  # 等待1分钟
            
            # 最终检查登录状态
            if "login" in driver.current_url.lower():
                print("❌ 登录超时，请检查登录状态")
                driver.quit()
                return False
        else:
            print("✅ 已经处于登录状态")
        
        # 等待页面完全加载
        print("⏳ 等待页面内容加载...")
        time.sleep(10)
        
        # 验证登录是否真正成功（检查页面中是否有登录后的特定元素）
        try:
            # 尝试查找登录后的页面元素，比如用户信息、订阅内容等
            # 这里可以根据实际页面结构调整选择器
            page_content = driver.page_source
            if "退出登录" in page_content or "logout" in page_content.lower() or "subscription" in page_content.lower():
                print("✅ 登录状态验证成功")
            else:
                print("⚠️  登录状态不确定，但继续保存Cookie")
        except Exception as e:
            print(f"⚠️  登录验证时出现警告: {e}")
        
        # 保存Cookie到文件
        cookies = driver.get_cookies()
        
        # 创建cookies目录（如果不存在）
        os.makedirs('cookies', exist_ok=True)
        
        # 保存Cookie文件
        cookie_file = 'cookies/tools_zhihu_cookies.pkl'
        with open(cookie_file, 'wb') as file:
            pickle.dump(cookies, file)
        
        print(f"✅ Cookie保存成功！文件: {cookie_file}")
        print(f"📊 保存了 {len(cookies)} 个Cookie")
        
        # 显示部分Cookie信息（不包含敏感信息）
        print("\n📋 Cookie摘要:")
        for i, cookie in enumerate(cookies[:5]):  # 只显示前5个
            print(f"  {i+1}. {cookie.get('name', 'N/A')} - 域名: {cookie.get('domain', 'N/A')}")
        
        if len(cookies) > 5:
            print(f"  ... 还有 {len(cookies) - 5} 个Cookie")
        
        # 测试Cookie是否有效：重新访问页面验证
        print("\n🧪 测试保存的Cookie是否有效...")
        test_cookie_access(driver, target_url)
        
        driver.quit()
        print("✅ Cookie保存流程完成！")
        return True
        
    except Exception as e:
        print(f"❌ 保存Cookie过程中出错: {e}")
        if 'driver' in locals():
            driver.quit()
        return False

def test_cookie_access(driver, url):
    """测试Cookie是否有效"""
    try:
        # 清除当前Cookie（模拟新会话）
        driver.delete_all_cookies()
        
        # 重新加载保存的Cookie
        cookie_file = 'cookies/tools_zhihu_cookies.pkl'
        if os.path.exists(cookie_file):
            with open(cookie_file, 'rb') as file:
                cookies = pickle.load(file)
            
            # 先访问域名根路径以设置Cookie
            driver.get("https://example.com")
            
            # 添加所有Cookie
            for cookie in cookies:
                # 处理可能的Cookie格式问题
                if 'sameSite' in cookie and cookie['sameSite'] not in ['Strict', 'Lax', 'None']:
                    cookie['sameSite'] = 'Lax'
                try:
                    driver.add_cookie(cookie)
                except Exception as e:
                    print(f"⚠️  添加Cookie {cookie.get('name')} 时警告: {e}")
            
            # 重新访问目标页面
            driver.get(url)
            time.sleep(5)
            
            # 检查是否成功访问（没有跳转到登录页面）
            if "login" not in driver.current_url.lower():
                print("✅ Cookie测试成功！可以免登录访问")
                return True
            else:
                print("❌ Cookie测试失败，仍需要登录")
                return False
    except Exception as e:
        print(f"❌ Cookie测试过程中出错: {e}")
        return False

# 执行保存Cookie
if __name__ == "__main__":
    print("🚀 开始保存Trendinsight网站Cookie...")
    success = save_cookies_for_trendinsight()
    
    if success:
        print("\n🎉 Cookie保存完成！您现在可以使用第二份代码来免登录访问了。")
    else:
        print("\n❌ Cookie保存失败，请检查网络连接和登录流程。")

🚀 开始保存Trendinsight网站Cookie...
✅ 浏览器初始化成功
🌐 正在访问Trendinsight网站...
🔗 当前URL: https://tools.zhihu.com/questionselection/buzzword
🔐 检测到登录页面，请手动完成登录...
💡 请在弹出的浏览器窗口中完成登录操作
⏳ 等待登录完成...
⏳ 仍在登录页面，请继续完成登录...
🔄 当前页面: https://tools.zhihu.com/questionselection/buzzword
⏳ 等待页面内容加载...
⚠️  登录状态不确定，但继续保存Cookie
✅ Cookie保存成功！文件: cookies/tools_zhihu_cookies.pkl
📊 保存了 11 个Cookie

📋 Cookie摘要:
  1. ll_b - 域名: tools.zhihu.com
  2. z_c0 - 域名: .zhihu.com
  3. captcha_ticket_v2 - 域名: .zhihu.com
  4. Hm_lpvt_98beee57fd2ef70ccdd5ca52b9740c49 - 域名: .zhihu.com
  5. _zap - 域名: .zhihu.com
  ... 还有 6 个Cookie

🧪 测试保存的Cookie是否有效...
⚠️  添加Cookie BEC 时警告: Message: invalid cookie domain: Cookie 'domain' mismatch
  (Session info: MicrosoftEdge=143.0.3650.80)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff621499435
	0x7ff621409aa4
	0x7ff6217f2d12
	0x7ff6212ae469
	0x7ff6212823ea
	0x7ff62125a2e5
	0x7ff62129c8de
	0x7ff62125982a
	0x7ff621258b33
	0x7ff621259653
	0x7ff621341d64
	0x7ff621350b1c
	0x7ff62134a6f